# Jaccard Index Matrix ( for comp only )

In [ ]:
import pandas as pd
import numpy as np
from jaccard_impl import get_authors_by_issn, jaccard_distance

# Charger le fichier CSV
df = pd.read_csv('./data/cleanScopus/clean_COMP.csv')

# Créer la carte ISSN -> auteurs
issn_authors_map = get_authors_by_issn(df)
issn_list = list(issn_authors_map.keys())

# Initialiser une matrice vide
num_issn = len(issn_list)
distance_matrix = np.zeros((num_issn, num_issn))

# Remplir la matrice avec les distances de Jaccard
for i in range(num_issn):
    for j in range(i, num_issn):  # On calcule uniquement la moitié supérieure (matrice symétrique)
        if i == j:
            distance_matrix[i][j] = 0  # Distance à soi-même est toujours 0
        else:
            set1 = issn_authors_map[issn_list[i]]
            set2 = issn_authors_map[issn_list[j]]
            distance = jaccard_distance(set1, set2)
            distance_matrix[i][j] = distance
            distance_matrix[j][i] = distance  # Matrice symétrique

# Convertir en DataFrame pour faciliter l'analyse
distance_df = pd.DataFrame(distance_matrix, index=issn_list, columns=issn_list)

# Sauvegarder dans un fichier CSV
output_file = './jaccardIndex/jaccard_COMP.csv'
distance_df.to_csv(output_file)

print(f"Matrice de distances de Jaccard sauvegardée dans : {output_file}")


initializing..
c'est parti pour le calcul !
0 / 1455
1 / 1455
2 / 1455
3 / 1455
4 / 1455
5 / 1455
6 / 1455
7 / 1455
8 / 1455
9 / 1455
10 / 1455
11 / 1455
12 / 1455
13 / 1455
14 / 1455
15 / 1455
16 / 1455
17 / 1455
18 / 1455
19 / 1455
20 / 1455
21 / 1455
22 / 1455
23 / 1455
24 / 1455
25 / 1455
26 / 1455
27 / 1455
28 / 1455
29 / 1455
30 / 1455
31 / 1455
32 / 1455
33 / 1455
34 / 1455
35 / 1455
36 / 1455
37 / 1455
38 / 1455
39 / 1455
40 / 1455
41 / 1455
42 / 1455
43 / 1455
44 / 1455
45 / 1455
46 / 1455
47 / 1455
48 / 1455
49 / 1455
50 / 1455
51 / 1455
52 / 1455
53 / 1455
54 / 1455
55 / 1455
56 / 1455
57 / 1455
58 / 1455
59 / 1455
60 / 1455
61 / 1455
62 / 1455
63 / 1455
64 / 1455
65 / 1455
66 / 1455
67 / 1455
68 / 1455
69 / 1455
70 / 1455
71 / 1455
72 / 1455
73 / 1455
74 / 1455
75 / 1455
76 / 1455
77 / 1455
78 / 1455
79 / 1455
80 / 1455
81 / 1455
82 / 1455
83 / 1455
84 / 1455
85 / 1455
86 / 1455
87 / 1455
88 / 1455
89 / 1455
90 / 1455
91 / 1455
92 / 1455
93 / 1455
94 / 1455
95 / 1455
96 / 1

## on affiche les stats

In [ ]:
import pandas as pd
import plotly.graph_objects as go

# Charger la matrice de distances Jaccard
distance_df = pd.read_csv('./jaccardIndex/jaccard_COMP.csv', index_col=0)

# Extraire les distances dans une liste
distances = distance_df.values.flatten()  # Convertir la matrice en une liste de distances

# Filtrer les valeurs pour calculer les proportions
total_distances = len(distances)
not_1_or_0 = sum(d!=0.0 and d!=1.0 for d in distances)
not_1_or_0_above_0_95 = sum(d!=0.0 and d!=1.0 and d>0.95 for d in distances)
not_1_or_0_under_0_95 = sum(d!=0.0 and d!=1.0 and d<0.95 for d in distances)
above_0_95 = sum(d > 0.95 for d in distances)
exactly_1 = sum(d == 1.0 for d in distances)
exactly_0 = sum(d == 0.0 for d in distances)

# Calculer les proportions
proportion_above_0_95 = above_0_95 / total_distances * 100
proportion_exactly_1 = exactly_1 / total_distances * 100
proportion_exactly_0 = exactly_0 / total_distances * 100

# Afficher les statistiques
print(f"Total de distances : {total_distances}")
print(f"Proportion de distances > 0.95 : {proportion_above_0_95:.2f}% ({above_0_95} distances)")
print(f"Proportion de distances = 1.0 : {proportion_exactly_1:.2f}% ({exactly_1} distances)")
print(f"Proportion de distances = 0.0 : {proportion_exactly_0:.2f}% ({exactly_0} distances)")

# Créer un pie chart avec Plotly
labels = ['between 1 and 0.95','between 0.95 and 0' ,'= 1.0', '= 0.0', 'Autres']
values = [
    not_1_or_0_above_0_95,
    not_1_or_0_under_0_95,
    exactly_1,
    exactly_0,
    total_distances - (not_1_or_0 + exactly_1 + exactly_0)
]

fig_pie = go.Figure(
    data=[go.Pie(labels=labels, values=values, hole=0.4)],
    layout=go.Layout(title="Proportions des distances de Jaccard")
)
fig_pie.show()

fig_gauge_0_95 = go.Figure(go.Indicator(
    mode="gauge+number",
    value=proportion_above_0_95,
    title={'text': "Proportion > 0.95"},
    gauge={'axis': {'range': [0, 100]}},
    number={'valueformat': '.5f'}  # 5 chiffres significatifs
))

fig_gauge_1_0 = go.Figure(go.Indicator(
    mode="gauge+number",
    value=proportion_exactly_1,
    title={'text': "Proportion = 1.0"},
    gauge={'axis': {'range': [0, 100]}},
    number={'valueformat': '.5f'}
))

fig_gauge_0_0 = go.Figure(go.Indicator(
    mode="gauge+number",
    value=proportion_exactly_0,
    title={'text': "Proportion = 0.0"},
    gauge={'axis': {'range': [0, 100]}},
    number={'valueformat': '.5f'}
))

# Afficher les gauge charts
fig_gauge_0_95.show()
fig_gauge_1_0.show()
fig_gauge_0_0.show()
fig_pie.write_html('./htmlplotlytest.html')

Total de distances : 3024121
Proportion de distances > 0.95 : 99.87% (3020264 distances)
Proportion de distances = 1.0 : 86.95% (2629612 distances)
Proportion de distances = 0.0 : 0.06% (1741 distances)


In [3]:
import pandas as pd
import plotly.graph_objects as go

# Créer un DataFrame pour résumer les proportions
gauge_data = {
    "Metric": [
        "Proportion > 0.95",
        "Proportion = 1.0",
        "Proportion = 0.0"
    ],
    "Value (%)": [
        proportion_above_0_95,
        proportion_exactly_1,
        proportion_exactly_0
    ]
}

gauge_df = pd.DataFrame(gauge_data)

# Créer une table Plotly
fig_table = go.Figure(data=[go.Table(
    header=dict(values=["Metric", "Value (%)"],
                fill_color='paleturquoise',
                align='left'),
    cells=dict(values=[gauge_df["Metric"], gauge_df["Value (%)"]],
               fill_color='lavender',
               align='left'))
])

fig_table.update_layout(title="Tableau des proportions des distances de Jaccard")
fig_table.show()


In [8]:
import pandas as pd
import numpy as np

# Charger la matrice de distances Jaccard
distance_df = pd.read_csv('./jaccardIndex/jaccard_COMP.csv', index_col=0)

# Étape 1 : Identifier les journaux avec une distance de 1 à tout le monde
journals_with_dist_1 = distance_df[(distance_df == 1.0).all(axis=1)].index.tolist()

# Nombre total de journaux
total_journals = len(distance_df)

# Nombre de journaux à distance 1
num_journals_dist_1 = len(journals_with_dist_1)

# Proportion de journaux à distance 1
proportion_journals_dist_1 = (num_journals_dist_1 / total_journals) * 100

print(f"Nombre total de journaux : {total_journals}")
print(f"Nombre de journaux à distance 1 : {num_journals_dist_1}")
print(f"Proportion de journaux à distance 1 : {proportion_journals_dist_1:.2f}%")

# Étape 2 : Générer des données fictives pour les publications et les auteurs
np.random.seed(42)  # Pour des résultats reproductibles
publications_df = pd.DataFrame({
    'Journal': distance_df.index,
    'Publications': np.random.randint(10, 500, size=total_journals),  # Valeurs aléatoires
    'Authors': np.random.randint(1, 50, size=total_journals)          # Valeurs aléatoires
})

# Étape 3 : Filtrer pour les journaux à distance 1
filtered_stats = publications_df[publications_df['Journal'].isin(journals_with_dist_1)]

# Afficher les résultats
print("Journaux à distance 1 et leurs statistiques :")
print(filtered_stats)

# Résumé des résultats
if not filtered_stats.empty:
    total_publications = filtered_stats['Publications'].sum()
    total_authors = filtered_stats['Authors'].sum()

    print(f"Nombre total de publications pour ces journaux : {total_publications}")
    print(f"Nombre total d’auteurs pour ces journaux : {total_authors}")
else:
    print("Aucun journal n'est à distance 1 de tout le monde.")


Nombre total de journaux : 1739
Nombre de journaux à distance 1 : 0
Proportion de journaux à distance 1 : 0.00%
Journaux à distance 1 et leurs statistiques :
Empty DataFrame
Columns: [Journal, Publications, Authors]
Index: []
Aucun journal n'est à distance 1 de tout le monde.


# maintenant que j'ai fais pour comp, je fais pour tout les autres 

In [13]:
import os
import pandas as pd
import numpy as np
from jaccard_impl import get_authors_by_issn, jaccard_distance

# Dossier contenant les fichiers clean_*.csv
data_folder = './data/cleanScopus'

# Lister les fichiers commençant par 'clean_'
files = [f for f in os.listdir(data_folder) if f.startswith('clean_') and f.endswith('.csv')]

# Fonction pour calculer la matrice de Jaccard
def compute_jaccard_matrix(file_path):
    # Charger le fichier CSV
    df = pd.read_csv(file_path)

    # Créer la carte ISSN -> auteurs
    issn_authors_map = get_authors_by_issn(df)
    issn_list = list(issn_authors_map.keys())

    # Initialiser une matrice vide
    num_issn = len(issn_list)
    distance_matrix = np.zeros((num_issn, num_issn))

    # Remplir la matrice avec les distances de Jaccard
    for i in range(num_issn):
        for j in range(i, num_issn):  # On calcule uniquement la moitié supérieure (matrice symétrique)
            if i == j:
                distance_matrix[i][j] = 0  # Distance à soi-même est toujours 0
            else:
                set1 = issn_authors_map[issn_list[i]]
                set2 = issn_authors_map[issn_list[j]]
                distance = jaccard_distance(set1, set2)
                distance_matrix[i][j] = distance
                distance_matrix[j][i] = distance  # Matrice symétrique

    # Convertir en DataFrame pour faciliter l'analyse
    distance_df = pd.DataFrame(distance_matrix, index=issn_list, columns=issn_list)
    return distance_df

# Calculer la matrice de Jaccard pour chaque fichier
output_folder = './jaccardIndex'
os.makedirs(output_folder, exist_ok=True)

overall_issn_authors_map = {}

for file in files:
    file_path = os.path.join(data_folder, file)
    print(f"Calcul de la matrice pour {file}...")

    # Calculer la matrice pour le fichier
    distance_df = compute_jaccard_matrix(file_path)

    # Sauvegarder dans un fichier CSV
    output_file = os.path.join(output_folder, f'jaccard_{file}')
    distance_df.to_csv(output_file)
    print(f"Matrice sauvegardée dans : {output_file}")

    # Ajouter les auteurs pour l'overall
    df = pd.read_csv(file_path)
    issn_authors_map = get_authors_by_issn(df)
    for issn, authors in issn_authors_map.items():
        if issn not in overall_issn_authors_map:
            overall_issn_authors_map[issn] = set()
        overall_issn_authors_map[issn].update(authors)

# Calculer la matrice "overall" (tous les fichiers combinés)
print("Calcul de la matrice globale (overall)...")
overall_issn_list = list(overall_issn_authors_map.keys())
num_issn_overall = len(overall_issn_list)
overall_distance_matrix = np.zeros((num_issn_overall, num_issn_overall))

for i in range(num_issn_overall):
    for j in range(i, num_issn_overall):
        if i == j:
            overall_distance_matrix[i][j] = 0  # Distance à soi-même
        else:
            set1 = overall_issn_authors_map[overall_issn_list[i]]
            set2 = overall_issn_authors_map[overall_issn_list[j]]
            distance = jaccard_distance(set1, set2)
            overall_distance_matrix[i][j] = distance
            overall_distance_matrix[j][i] = distance  # Matrice symétrique

# Convertir et sauvegarder la matrice "overall"
overall_distance_df = pd.DataFrame(overall_distance_matrix, index=overall_issn_list, columns=overall_issn_list)
overall_output_file = os.path.join(output_folder, 'jaccard_overall.csv')
overall_distance_df.to_csv(overall_output_file)
print(f"Matrice globale sauvegardée dans : {overall_output_file}")


initializing..
c'est parti pour le calcul !
0 / 1455
1 / 1455
2 / 1455
3 / 1455
4 / 1455
5 / 1455
6 / 1455
7 / 1455
8 / 1455
9 / 1455
10 / 1455
11 / 1455
12 / 1455
13 / 1455
14 / 1455
15 / 1455
16 / 1455
17 / 1455
18 / 1455
19 / 1455
20 / 1455
21 / 1455
22 / 1455
23 / 1455
24 / 1455
25 / 1455
26 / 1455
27 / 1455
28 / 1455
29 / 1455
30 / 1455
31 / 1455
32 / 1455
33 / 1455
34 / 1455
35 / 1455
36 / 1455
37 / 1455
38 / 1455
39 / 1455
40 / 1455
41 / 1455
42 / 1455
43 / 1455
44 / 1455
45 / 1455
46 / 1455
47 / 1455
48 / 1455
49 / 1455
50 / 1455
51 / 1455
52 / 1455
53 / 1455
54 / 1455
55 / 1455
56 / 1455
57 / 1455
58 / 1455
59 / 1455
60 / 1455
61 / 1455
62 / 1455
63 / 1455
64 / 1455
65 / 1455
66 / 1455
67 / 1455
68 / 1455
69 / 1455
70 / 1455
71 / 1455
72 / 1455
73 / 1455
74 / 1455
75 / 1455
76 / 1455
77 / 1455
78 / 1455
79 / 1455
80 / 1455
81 / 1455
82 / 1455
83 / 1455
84 / 1455
85 / 1455
86 / 1455
87 / 1455
88 / 1455
89 / 1455
90 / 1455
91 / 1455
92 / 1455
93 / 1455
94 / 1455
95 / 1455
96 / 1

C:\Users\mezio\AppData\Local\Temp\ipykernel_27720\789121955.py:15: DtypeWarning:

Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.



Matrice sauvegardée dans : ./jaccardIndex\jaccard_clean_HEAL.csv


C:\Users\mezio\AppData\Local\Temp\ipykernel_27720\789121955.py:60: DtypeWarning:

Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.



Calcul de la matrice pour clean_IMMU.csv...
Matrice sauvegardée dans : ./jaccardIndex\jaccard_clean_IMMU.csv
Calcul de la matrice pour clean_MATE.csv...
Matrice sauvegardée dans : ./jaccardIndex\jaccard_clean_MATE.csv
Calcul de la matrice pour clean_MATH.csv...
Matrice sauvegardée dans : ./jaccardIndex\jaccard_clean_MATH.csv
Calcul de la matrice pour clean_MULT.csv...
Matrice sauvegardée dans : ./jaccardIndex\jaccard_clean_MULT.csv
Calcul de la matrice pour clean_NEUR.csv...
Matrice sauvegardée dans : ./jaccardIndex\jaccard_clean_NEUR.csv
Calcul de la matrice pour clean_NURS.csv...
Matrice sauvegardée dans : ./jaccardIndex\jaccard_clean_NURS.csv
Calcul de la matrice pour clean_PHAR.csv...
Matrice sauvegardée dans : ./jaccardIndex\jaccard_clean_PHAR.csv
Calcul de la matrice pour clean_PHYS.csv...
Matrice sauvegardée dans : ./jaccardIndex\jaccard_clean_PHYS.csv
Calcul de la matrice pour clean_PSYC.csv...
Matrice sauvegardée dans : ./jaccardIndex\jaccard_clean_PSYC.csv
Calcul de la matric

In [17]:
import os
import pandas as pd
import plotly.graph_objects as go

# Dossier contenant les fichiers Jaccard
jaccard_folder = './jaccardIndex/'
results_folder = './results/'

# Vérifier et créer le dossier `results` si nécessaire
os.makedirs(results_folder, exist_ok=True)

# Lister tous les fichiers CSV dans le dossier
jaccard_files = [file for file in os.listdir(jaccard_folder) if file.endswith('.csv')]

# Boucle à travers tous les fichiers
for jaccard_file in jaccard_files:
    file_path = os.path.join(jaccard_folder, jaccard_file)

    # Charger la matrice de distances Jaccard
    distance_df = pd.read_csv(file_path, index_col=0)

    # Extraire les distances dans une liste
    distances = distance_df.values.flatten()

    # Filtrer les valeurs pour calculer les proportions
    total_distances = len(distances)
    not_1_or_0 = sum(d != 0.0 and d != 1.0 for d in distances)
    not_1_or_0_above_0_95 = sum(d != 0.0 and d != 1.0 and d > 0.95 for d in distances)
    not_1_or_0_under_0_95 = sum(d != 0.0 and d != 1.0 and d < 0.95 for d in distances)
    above_0_95 = sum(d > 0.95 for d in distances)
    exactly_1 = sum(d == 1.0 for d in distances)
    exactly_0 = sum(d == 0.0 for d in distances)

    # Calculer les proportions
    proportion_above_0_95 = above_0_95 / total_distances * 100
    proportion_exactly_1 = exactly_1 / total_distances * 100
    proportion_exactly_0 = exactly_0 / total_distances * 100

    # Créer un pie chart avec Plotly
    labels = ['between 1 and 0.95', 'between 0.95 and 0', '= 1.0', '= 0.0', 'Autres']
    values = [
        not_1_or_0_above_0_95,
        not_1_or_0_under_0_95,
        exactly_1,
        exactly_0,
        total_distances - (not_1_or_0 + exactly_1 + exactly_0)
    ]

    fig_pie = go.Figure(
        data=[go.Pie(labels=labels, values=values, hole=0.4)],
        layout=go.Layout(title=f"Proportions des distances de Jaccard ({jaccard_file})")
    )
    # Nom du fichier pour sauvegarder
    pie_file = os.path.join(results_folder, f'{os.path.splitext(jaccard_file)[0]}_pie.html')
    fig_pie.write_html(pie_file)

    # Créer des gauges
    fig_gauge_0_95 = go.Figure(go.Indicator(
        mode="gauge+number",
        value=proportion_above_0_95,
        title={'text': "Proportion > 0.95"},
        gauge={'axis': {'range': [0, 100]}},
        number={'valueformat': '.5f'}
    ))
    fig_gauge_0_95.write_html(os.path.join(results_folder, f'{os.path.splitext(jaccard_file)[0]}_gauge_0_95.html'))

    fig_gauge_1_0 = go.Figure(go.Indicator(
        mode="gauge+number",
        value=proportion_exactly_1,
        title={'text': "Proportion = 1.0"},
        gauge={'axis': {'range': [0, 100]}},
        number={'valueformat': '.5f'}
    ))
    fig_gauge_1_0.write_html(os.path.join(results_folder, f'{os.path.splitext(jaccard_file)[0]}_gauge_1_0.html'))

    fig_gauge_0_0 = go.Figure(go.Indicator(
        mode="gauge+number",
        value=proportion_exactly_0,
        title={'text': "Proportion = 0.0"},
        gauge={'axis': {'range': [0, 100]}},
        number={'valueformat': '.5f'}
    ))
    fig_gauge_0_0.write_html(os.path.join(results_folder, f'{os.path.splitext(jaccard_file)[0]}_gauge_0_0.html'))

print("Traitement terminé pour tous les fichiers.")


C:\Users\mezio\AppData\Local\Temp\ipykernel_27720\2212370079.py:20: DtypeWarning:

Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.



Traitement terminé pour tous les fichiers.


In [ ]:
import os
import pandas as pd
import numpy as np

# Dossier contenant les fichiers Jaccard
jaccard_folder = './jaccardIndex/'
results_folder = './results/'

# Vérifier et créer le dossier `results` si nécessaire
os.makedirs(results_folder, exist_ok=True)

# Lister tous les fichiers CSV dans le dossier
jaccard_files = [file for file in os.listdir(jaccard_folder) if file.endswith('.csv')]

# Boucle à travers tous les fichiers
for jaccard_file in jaccard_files:
    try:
        file_path = os.path.join(jaccard_folder, jaccard_file)

        # Charger la matrice de distances Jaccard
        distance_df = pd.read_csv(file_path, index_col=0)

        # Récupérer la diagonale (distance des journaux à eux-mêmes)
        diagonal = np.diag(distance_df.values)

        # Identifier les journaux qui ont au moins une distance entre 0 et 1
        has_between_0_and_1 = (distance_df > 0) & (distance_df < 1)
        journals_with_between_0_and_1 = has_between_0_and_1.any(axis=1)

        # Identifier les journaux qui sont à distance 0 avec eux-mêmes
        self_distance_0 = diagonal == 0

        # Journaux répondant aux critères
        valid_journals = distance_df.index[journals_with_between_0_and_1 & self_distance_0].tolist()

        # Nombre total de journaux
        total_journals = len(distance_df)

        # Nombre de journaux répondant au critère
        num_valid_journals = len(valid_journals)

        # Proportion de journaux répondant au critère
        proportion_valid_journals = (num_valid_journals / total_journals) * 100

        # Afficher les statistiques dans la console
        print(f"Fichier : {jaccard_file}")
        print(f"Nombre total de journaux : {total_journals}")
        print(f"Nombre de journaux ayant des distances entre 0 et 1 : {num_valid_journals}")
        print(f"Proportion de journaux ayant des distances entre 0 et 1 : {proportion_valid_journals:.2f}%")

        # Étape 2 : Générer des données fictives pour les publications et les auteurs
        np.random.seed(42)  # Pour des résultats reproductibles
        publications_df = pd.DataFrame({
            'Journal': distance_df.index,
            'Publications': np.random.randint(10, 500, size=total_journals),
            'Authors': np.random.randint(1, 50, size=total_journals)
        })

        # Étape 3 : Filtrer pour les journaux répondant au critère
        filtered_stats = publications_df[publications_df['Journal'].isin(valid_journals)]

        # Sauvegarder les résultats dans un fichier CSV
        output_file = os.path.join(results_folder, f'{os.path.splitext(jaccard_file)[0]}_filtered_stats.csv')
        filtered_stats.to_csv(output_file, index=False)

        # Résumé des résultats
        if not filtered_stats.empty:
            total_publications = filtered_stats['Publications'].sum()
            total_authors = filtered_stats['Authors'].sum()

            print(f"Nombre total de publications pour ces journaux : {total_publications}")
            print(f"Nombre total d’auteurs pour ces journaux : {total_authors}")
        else:
            print("Aucun journal ne correspond aux critères.")

    except Exception as e:
        print(f"Erreur lors du traitement du fichier {jaccard_file}: {e}")



Fichier : jaccard_clean_AGRI.csv
Nombre total de journaux : 2262
Nombre de journaux ayant des distances entre 0 et 1 : 2218
Proportion de journaux ayant des distances entre 0 et 1 : 98.05%
Nombre total de publications pour ces journaux : 561182
Nombre total d’auteurs pour ces journaux : 55583
Fichier : jaccard_clean_ARTS.csv
Nombre total de journaux : 3670
Nombre de journaux ayant des distances entre 0 et 1 : 3241
Proportion de journaux ayant des distances entre 0 et 1 : 88.31%
Nombre total de publications pour ces journaux : 827964
Nombre total d’auteurs pour ces journaux : 82324
Fichier : jaccard_clean_BIOC.csv
Nombre total de journaux : 2411
Nombre de journaux ayant des distances entre 0 et 1 : 2390
Proportion de journaux ayant des distances entre 0 et 1 : 99.13%
Nombre total de publications pour ces journaux : 606345
Nombre total d’auteurs pour ces journaux : 60209
Fichier : jaccard_clean_BUSI.csv
Nombre total de journaux : 1369
Nombre de journaux ayant des distances entre 0 et 1 :